In [1]:
import numpy as np
import string
from utils import Kron

In [2]:
def TupleToString(tup):
    # Convert a tuple to a string where each number is assigned to its string form.
    return "".join(list(map(lambda i: string.printable[10 + i], tup)))

In [3]:
def YieldScalar(tndict):
    # Check if contracting all the tensors in the dictionary yields a scalar.
    supports = np.array([np.array(list(support), dtype = np.int) for (support, __) in tndict], dtype = np.int).flatten()
    #print("supports = {}".format(supports))
    visited = np.zeros(supports.shape[0], dtype = np.int)
    for i in range(supports.size):
        not_scalar = 1
        for j in range(supports.size):
            if (i != j):
                if (supports[i] == supports[j]):
                    not_scalar = 0
                    #print("Comparing support[%d] = %d and support[%d] = %d." % (i, supports[i], j, supports[j])) 
    scalar = 1 - not_scalar
    return scalar

In [4]:
def TensorTrace(tndict, opt = "greedy"):
    # Contract a set of tensors provided as a dictionary.
    # The result of the traction must be a number, in other words, there should be no free indices.
    if (YieldScalar(tndict) == 0):
        print("The Tensor contraction does not result in a trace.")
        return None
    # We want to use np.einsum(...) to perform the Tensor contraction.
    # Every element of the input dictionary is a tuple, containing the support of a tensor and its operator form.
    # Numpy's einsum function simply needs the Tensors and the corresponding labels associated to its indices.
    # We will convert the support of each Tensor to a string, that serves as its label.
    scheme = ",".join([TupleToString(support) for (support, __) in tndict])
    print("Contraction scheme: {}".format(scheme))
    ops = [op for (__, op) in tndict]
    ops_args = ", ".join([("ops[%d]" % d) for d in range(len(ops))])
    print("np.einsum_path(\'%s->\', %s, optimize=\'%s\')" % (scheme, ops_args, opt))
    path = eval("np.einsum_path(\'%s->\', %s, optimize=\'%s\')" % (scheme, ops_args, opt))
    print("Contraction process\n{}: {}\n{}".format(path[0][0], path[0][1:], path[1]))
    trace = np.einsum(scheme, *ops, optimize=path[0])
    #print("Trace = {}.".format(trace))
    return trace

In [5]:
N = 24
dims = 4
bond = 4
# create some numpy tensors
tensors = [np.random.rand(*[bond]*dims) for __ in range(0, N, 2)]
# labels should ensure that there is no free index.
labels = [tuple([(i + j) % N for j in range(4)]) for i in range(0, N, 2)]
print(labels)
# Prepare the dictionary.
tndict = [(labels[i], tensors[i]) for i in range(len(labels))]
#print([tnop for (lab, tnop) in tndict])

[(0, 1, 2, 3), (2, 3, 4, 5), (4, 5, 6, 7), (6, 7, 8, 9), (8, 9, 10, 11), (10, 11, 12, 13), (12, 13, 14, 15), (14, 15, 16, 17), (16, 17, 18, 19), (18, 19, 20, 21), (20, 21, 22, 23), (22, 23, 0, 1)]


In [6]:
trace = TensorTrace(tndict, opt="greedy")
print("Trace = {}.".format(trace))

Contraction scheme: abcd,cdef,efgh,ghij,ijkl,klmn,mnop,opqr,qrst,stuv,uvwx,wxab
np.einsum_path('abcd,cdef,efgh,ghij,ijkl,klmn,mnop,opqr,qrst,stuv,uvwx,wxab->', ops[0], ops[1], ops[2], ops[3], ops[4], ops[5], ops[6], ops[7], ops[8], ops[9], ops[10], ops[11], optimize='greedy')
Contraction process
einsum_path: [(0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1), (0, 1)]
  Complete contraction:  abcd,cdef,efgh,ghij,ijkl,klmn,mnop,opqr,qrst,stuv,uvwx,wxab->
         Naive scaling:  24
     Optimized scaling:  6
      Naive FLOP count:  3.378e+15
  Optimized FLOP count:  8.243e+04
   Theoretical speedup:  40975091535.282
  Largest intermediate:  2.560e+02 elements
--------------------------------------------------------------------------
scaling                  current                                remaining
--------------------------------------------------------------------------
   6             cdef,abcd->abef efgh,ghij,ijkl,klmn,mnop,opqr,qrst,stuv,uvwx,wx

In [7]:
def OptimalEinsum(scheme, ops, opt = "greedy"):
    # Contract a tensor network using einsum supplemented with its optimization tools.
    ops_args = ", ".join([("ops[%d]" % d) for d in range(len(ops))])
    print("Calling np.einsum({}, {})\nwhere shapes are\n{}.".format(scheme, ops_args, [op.shape for op in ops]))
    path = eval("np.einsum_path(\'%s\', %s, optimize=\'%s\')" % (scheme, ops_args, opt))
    #print("Contraction process\n{}: {}\n{}".format(path[0][0], path[0][1:], path[1]))
    prod = np.einsum(scheme, *ops, optimize=path[0])
    return prod

In [8]:
def TensorTranspose(tensor):
    # Transpose the tensor, in other words, exchange its row and column indices.
    # Note that when we reshape a matrix into (D, D, ..., D) tensor, it stores the indices as as
    # row_1, row_2, row_3, ..., row_(D/2), col_1, ..., col_(D/2).
    rows = range(0, tensor.ndim//2)
    cols = range(tensor.ndim//2, tensor.ndim)
    tp_indices = np.concatenate((cols, rows))
    return np.transpose(tensor, tp_indices)

In [9]:
# Testing TensorTranspose
nq = 2
tensor = np.reshape(np.random.rand(2**nq, 2**nq), [2, 2]*nq)
tp_tensor = TensorTranspose(tensor)
#print("tensor\n{}\nand its transpose\n{}".format(tensor.reshape(2**dims, 2**dims), tp_tensor.reshape(2**dims, 2**dims)))
print("A ?= A.T is {}".format(np.allclose(tensor.reshape(2**nq, 2**nq), tp_tensor.reshape(2**nq, 2**nq).T)))

A ?= A.T is True


In [10]:
def GetNQubitPauli(ind, nq):
    # Compute the n-qubit Pauli that is at position 'i' in an ordering based on [I, X, Y, Z].
    # We will express the input number in base 4^n - 1.
    pauli = np.zeros(nq, dtype = np.int)
    for i in range(nq):
        pauli[i] = ind % 4
        ind = int(ind//4)
    return pauli

In [11]:
# Testing GetNQubitPauli
GetNQubitPauli(172, 4)

array([0, 3, 2, 2])

In [12]:
def TensorKron(tn1, tn2):
    # Compute the Kronecker product of two tensors A and B.
    # This is not equal to np.tensordot(A, B, axis = 0), see: https://stackoverflow.com/questions/52125078/why-does-tensordot-reshape-not-agree-with-kron.
    # Note that when we reshape a matrix into (D, D, ..., D) tensor, it stores the indices as as
    # row_1, row_2, row_3, ..., row_(D/2), col_1, ..., col_(D/2).
    # We will implement Kronecker product using einsum, as
    # np.einsum('rA1 rA2 .. rAn, cA1 cA2 .. cAn, rB1 rB2 .. rBn, cB1 cB2 .. cBn -> rA1 rB1 rA2 rB2 .. rAn rBn, cA1 cB1 cA2 cB2 .. cAn cBn', A, B).
    if (tn1.ndim != tn2.ndim):
        print("TensorKron does not work presently for tensors of different dimensions.")
        return None
    tn1_rows = [string.printable[10 + i] for i in range(tn1.ndim//2)]
    tn1_cols = [string.printable[10 + i] for i in range(tn1.ndim//2, tn1.ndim)]
    tn2_rows = [string.printable[10 + tn1.ndim + i] for i in range(tn2.ndim//2)]
    tn2_cols = [string.printable[10 + tn1.ndim + i] for i in range(tn2.ndim//2, tn2.ndim)]
    #kron_inds = ["%s%s" % (tn1_rows[i], tn2_rows[i]) for i in range(tn1.ndim//2)]
    #kron_inds += ["%s%s" % (tn1_cols[i], tn2_cols[i]) for i in range(tn1.ndim//2)]
    kron_inds = ["%s" % (tn1_rows[i]) for i in range(tn1.ndim//2)]
    kron_inds += ["%s" % (tn2_rows[i]) for i in range(tn1.ndim//2)]
    kron_inds += ["%s" % (tn1_cols[i]) for i in range(tn1.ndim//2)]
    kron_inds += ["%s" % (tn2_cols[i]) for i in range(tn1.ndim//2)]
    scheme = ("%s%s,%s%s->%s" % ("".join(tn1_rows), "".join(tn1_cols), "".join(tn2_rows), "".join(tn2_cols), "".join(kron_inds)))
    return OptimalEinsum(scheme, [tn1, tn2], opt = "greedy")

In [13]:
# Testing TensorKron
nq = 4
tn1 = np.reshape(np.random.rand(2**nq, 2**nq), [2, 2]*nq)
tn2 = np.reshape(np.random.rand(2**nq, 2**nq), [2, 2]*nq)
kn_tensor = TensorKron(tn1, tn2)
#print("A\n{}\nB\n{}\nA o B = \n{}".format(tn1.reshape(2**nq, 2**nq), tn2.reshape(2**nq, 2**nq), kn_tensor.reshape(4**nq, 4**nq)))
print("TensorKron(A, B) ?= A o B is {}".format(np.allclose(kn_tensor.reshape(4**nq, 4**nq), np.kron(tn1.reshape(2**nq, 2**nq), tn2.reshape(2**nq, 2**nq)))))

Calling np.einsum(abcdefgh,ijklmnop->abcdijklefghmnop, ops[0], ops[1])
where shapes are
[(2, 2, 2, 2, 2, 2, 2, 2), (2, 2, 2, 2, 2, 2, 2, 2)].
TensorKron(A, B) ?= A o B is True


In [14]:
def TraceDot(tn1, tn2):
    # Compute the trace of the dot product of two tensors A and B.
    # If the indices of A are i_0 i_1, ..., i_(2n-1) and that of B are j_0 j_1 ... j_(2n-1)
    # then want to contract the indices i_(2k) with j_(2k+1), for all k in [0, n-1].
    # While calling np.einsum, we need to ensure that the row index of A is equal to the column index of B.
    # Additionally to ensure that we have a trace, we need to match the row and column indices of the product.
    tn1_rows = [string.printable[10 + i] for i in range(tn1.ndim//2)]
    tn1_cols = [string.printable[10 + i] for i in range(tn1.ndim//2, tn1.ndim)]
    # The column indices of tn1 should match row indices of tn2
    # So, tn1_cols = tn2_rows.
    # the row and column indices of the product must match
    # So, tn1_rows = tn2_cols.
    scheme = ("%s%s,%s%s->" % ("".join(tn1_rows), "".join(tn1_cols), "".join(tn1_cols), "".join(tn1_rows)))
    return OptimalEinsum(scheme, [tn1, tn2], opt = "greedy")

In [15]:
# Testing TraceDot
nq = 5
tn1 = np.reshape(np.random.rand(2**nq, 2**nq), [2, 2]*nq)
tn2 = np.reshape(np.random.rand(2**nq, 2**nq), [2, 2]*nq)
trdot = TraceDot(tn1, tn2)
nptr = np.trace(np.dot(tn1.reshape(2**nq, 2**nq), tn2.reshape(2**nq, 2**nq)))
#print("A\n{}\nB\n{}\nTr(A . B) = \n{}".format(tn1.reshape(2**nq, 2**nq), tn2.reshape(2**nq, 2**nq), trdot))
#print("TensorTrace = {}\nNumpy Trace = {}".format(trdot, nptr))
print("TraceDot(A, B) ?= Tr(A . B) is {}.".format(np.allclose(trdot, nptr)))

Calling np.einsum(abcdefghij,fghijabcde->, ops[0], ops[1])
where shapes are
[(2, 2, 2, 2, 2, 2, 2, 2, 2, 2), (2, 2, 2, 2, 2, 2, 2, 2, 2, 2)].
TraceDot(A, B) ?= Tr(A . B) is True.


In [16]:
def PauliTensor(pauli_op):
    # Convert a Pauli in operator form to a tensor.
    # The tensor product of A B .. Z is given by simply putting the rows indices of A, B, ..., Z together, followed by their column indices.
    # Each qubit index q can be assigned a pair of labels for the row and columns of the Pauli matrix on q: C[2q], C[2q + 1].
    # Pauli matrices
    characters = string.printable[10:]
    # replace the following line with the variable from globalvars.py
    Pauli = np.array([[[1, 0], [0, 1]], [[0, 1], [1, 0]], [[0, -1j], [1j, 0]], [[1, 0], [0, -1]]], dtype=np.complex128)
    nq = pauli_op.shape[0]
    labels = ",".join(["%s%s" % (characters[2 * q], characters[2 * q + 1]) for q in range(nq)])
    ops = [Pauli[pauli_op[q], :, :] for q in range(nq)]
    kn_indices = ["%s" % (characters[2 * q]) for q in range(nq)]
    kn_indices += ["%s" % (characters[2 * q + 1]) for q in range(nq)]
    kn_label = "".join(kn_indices)
    scheme = "%s->%s" % (labels, kn_label)
    pauli_tensor = OptimalEinsum(scheme, ops)
    return pauli_tensor
    

In [17]:
N = 3
pauli_op = np.random.randint(0, high=4, size=(N,))
print("Pauli operator: {}".format(pauli_op))
tn_pauli = PauliTensor(pauli_op)
Pauli = np.array([[[1, 0], [0, 1]], [[0, 1], [1, 0]], [[0, -1j], [1j, 0]], [[1, 0], [0, -1]]], dtype=np.complex128)
np_pauli = Kron(*Pauli[pauli_op, :, :])
print("PauliTensor - Numpy = {}".format(np.allclose(tn_pauli.reshape(2**N, 2**N), np_pauli)))

Pauli operator: [1 3 3]
Calling np.einsum(ab,cd,ef->acebdf, ops[0], ops[1], ops[2])
where shapes are
[(2, 2), (2, 2), (2, 2)].
PauliTensor - Numpy = True


In [18]:
def KraussToTheta(kraus):
    # Convert from the Kraus representation to the "Theta" representation.
    # The "Theta" matrix T of a CPTP map whose chi-matrix is X is defined as:
    # T_ij = \sum_(ij) [ X_ij (P_i o (P_j)^T) ]
    # Note that the chi matrix X can be defined using the Kraus matrices {K_k} in the following way
    # X_ij = \sum_k [ <P_i|K_k><K_k|P_j> ]
    # 	   = \sum_k [ Tr(P_i K_k) Tr((K_k)^\dag P_j)]
    # So we find that
    # T = \sum_(ij) [ \sum_k [ Tr(P_i K_k) Tr((K_k)^\dag P_j)] ] (P_i o (P_j)^T) ]
    # We will store T as a Tensor with dimension = (2 * number of qubits) and bond dimension = 4.
    nq = int(np.log2(kraus.shape[1]))
    theta = np.zeros(tuple([4, 4]*nq), dtype = np.complex128)
    for i in range(4**nq):
        for j in range(4**nq):
            Pi = PauliTensor(GetNQubitPauli(i, nq))
            Pj = PauliTensor(GetNQubitPauli(j, nq))
            PjT = TensorTranspose(Pj)
            PioPjT = np.reshape(TensorKron(Pi, PjT), tuple([4, 4] * nq))
            #print("Pi.shape = {}\nPi\n{}".format(Pi.shape, Pi))
            #print("Pj.shape = {}\nPj\n{}".format(Pj.shape, Pj))
            chi_ij = 0 + 0 * 1j
            for k in range(kraus.shape[0]):
                K = np.reshape(kraus[k, :, :], tuple([2, 2]*nq))
                Kdag = np.conj(TensorTranspose(K))
                #print("K.shape = {}\nK\n{}".format(K.shape, K))
                #print("TraceDot(K, Pi) = {}".format(TraceDot(K, Pi)))
                #print("Kdag.shape = {}\nKdag\n{}".format(Kdag.shape, Kdag))
                #print("Pj.shape = {}\nPj\n{}".format(Pj.shape, Pj))
                #print("TraceDot(Kdag, Pj) = {}".format(TraceDot(K, Pj)))
                chi_ij += TraceDot(Pi, K) * TraceDot(Pj, Kdag)
            chi_ij /= 4**nq
            #print("Chi[%d, %d] = %g + i %g" % (i, j, np.real(coeff), np.imag(coeff)))
            theta += chi_ij * PioPjT
    return theta

In [19]:
# depolarizing channel
N = 1
Pauli = np.array([[[1, 0], [0, 1]], [[0, 1], [1, 0]], [[0, -1j], [1j, 0]], [[1, 0], [0, -1]]], dtype=np.complex128)
kraus_dp = np.zeros((4**N, 2**N, 2**N), dtype = np.complex128)
rate = 0.1
kraus_dp[0, :, :] = np.sqrt(1 - rate) * Pauli[0, :, :]
for k in range(1, 4):
    kraus_dp[k, :, :] = np.sqrt(rate/3) * Pauli[k, :, :]
theta = KraussToTheta(kraus_dp)
print("Theta\n{}".format(theta))

Calling np.einsum(ab->ab, ops[0])
where shapes are
[(2, 2)].
Calling np.einsum(ab->ab, ops[0])
where shapes are
[(2, 2)].
Calling np.einsum(ab,cd->acbd, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab->ab, ops[0])
where shapes are
[(2, 2)].
Calling np.einsum(ab->ab, ops[0])
where shapes are
[(2, 2)].
Calling np.einsum(ab,cd->acbd, ops[0], ops[1])
whe

[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab->ab, ops[0])
where shapes are
[(2, 2)].
Calling np.einsum(ab->ab, ops[0])
where shapes are
[(2, 2)].
Calling np.einsum(ab,cd->acbd, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)].
Calling np.einsum(ab,ba->, ops[0], ops[1])
where shapes are
[(2, 2), (2, 2)]

In [52]:
def SupportToLabel(supports, characters = None):
    # Convert a list of qubit indices to labels for a tensor.
    # Each qubit index corresponds to a pair of labels, indicating the row and column indices of the 2 x 2 matrix which acts non-trivially on that qubit.
    # Each number in the support is mapped to a pair of alphabets in the characters list, as: x -> (characters[2x], characters[2x + 1]).
    # Eg. (x, y, z) ---> (C[2x] C[2y] C[2z] , C[2x + 1] C[2y + 1] C[2z + 1])
    if characters == None:
        characters = [c for c in string.ascii_lowercase] + [c for c in string.ascii_uppercase]
    #print("characters\n{}".format(characters))
    #print("support\n{}".format(supports))
    labels = [[[-1, -1] for q in interac] for interac in supports]
    #print("labels\n{}".format(labels))
    unique_qubits = np.unique([q for sup in supports for q in sup])
    #print("unique qubits\n{}".format(unique_qubits))
    free_index = {q:[-1, -1] for q in unique_qubits}
    for i in range(len(supports)):
            sup = supports[i]
            #print("Support: {}".format(sup))
            for j in range(len(sup)):
                    #print("Qubit: {}".format(sup[j]))
                    q = sup[j]
                    if (free_index[q][0] == -1):
                        free_index[q][0] = characters.pop()
                        free_index[q][1] = characters.pop()
                        #print("Assigning {} and {} to qubit {} of map {}\n".format(free_index[q][0],free_index[q][1],q,i))
                        labels[i][j][0] = free_index[q][0]
                        labels[i][j][1] = free_index[q][1]
                    else:
                        labels[i][j][0] = free_index[q][1]
                        free_index[q][1] = characters.pop()
                        labels[i][j][1] = free_index[q][1]
                        #print("Assigning {} and {} to qubit {} of map {}\n".format(labels[i][j][0],labels[i][j][1],q,i))
                    #print("labels\n{}\nfree index\n{}".format(labels, free_index))
    #print("labels\n{}\nfree index\n{}".format(labels, free_index))
    return (labels, free_index)

In [57]:
def ContractThetaNetwork(theta_dict):
    # Compute the Theta matrix of a composition of channels.
    # The individual channels are provided a list where each one is a pair: (s, O) where s is the support and O is the theta matrix.
    # We will use einsum to contract the tensor network of channels.
    supports = [list(sup) for (sup, op) in theta_dict]
    (contraction_labels, free_labels) = SupportToLabel(supports)
    print("contraction_labels = {}".format(contraction_labels))
    row_labels = ["".join([q[0] for q in interac]) for interac in contraction_labels]
    print("row_contraction_labels = {}".format(row_labels))
    col_labels = ["".join([q[1] for q in interac]) for interac in contraction_labels]
    print("col_contraction_labels = {}".format(col_labels))
    contraction_scheme = "%s,%s" % ("".join(row_labels), "".join(col_labels))
    print("contraction_scheme = {}".format(contraction_scheme))
    free_row_labels = [free_labels[q][0] for q in free_labels]
    print("free_row_labels = {}".format(free_row_labels))
    free_col_labels = [free_labels[q][1] for q in free_labels]
    print("free_col_labels = {}".format(free_col_labels))
    contracted_labels = "%s,%s" % ("".join(free_row_labels), "".join(free_col_labels))
    print("contracted_labels = {}".format(contracted_labels))
    scheme = "%s->%s" % (contraction_scheme, contracted_labels)
    print("Contraction scheme = {}".format(scheme))
    theta_ops = [op for (__, op) in theta_dict]
    composed = OptimalEinsum(contraction_scheme, theta_ops)
    return composed

In [61]:
theta_dict = [((1,3), np.random.rand(4,4,4,4)), ((0,2), np.random.rand(4,4,4,4))]
#print("Contracting the Theta network\n{}".format(theta_dict))
contracted = ContractThetaNetwork(theta_dict)
print("Result has dimensions {}\n{}".format(contracted.shape, contracted))

contraction_labels = [[['Z', 'Y'], ['X', 'W']], [['V', 'U'], ['T', 'S']]]
row_contraction_labels = ['ZX', 'VT']
col_contraction_labels = ['YW', 'US']
contraction_scheme = ZXVT,YWUS
free_row_labels = ['V', 'Z', 'T', 'X']
free_col_labels = ['U', 'Y', 'S', 'W']
contracted_labels = VZTX,UYSW
Contraction scheme = ZXVT,YWUS->VZTX,UYSW
Calling np.einsum(ZXVT,YWUS, ops[0], ops[1])
where shapes are
[(4, 4, 4, 4), (4, 4, 4, 4)].
Result has dimensions (4, 4, 4, 4, 4, 4, 4, 4)
[[[[[[[[2.72375012e-01 6.33407766e-02 1.77118854e-02 2.02053430e-01]
       [3.80243484e-01 8.84255769e-02 2.47263100e-02 2.82072498e-01]
       [3.86180163e-03 8.98061509e-04 2.51123578e-04 2.86476449e-03]
       [2.44781544e-01 5.69239188e-02 1.59175491e-02 1.81584023e-01]]

      [[3.50558277e-01 3.68661012e-01 1.68065788e-01 2.21256511e-01]
       [4.89389608e-01 5.14661556e-01 2.34624755e-01 3.08880561e-01]
       [4.97030367e-03 5.22696882e-03 2.38287913e-03 3.13703062e-03]
       [3.15044304e-01 3.31313107e-01 1.51039